In [0]:
# filepaths
top50genesPaper_SPM = '/content/drive/My Drive/Proj Senior/top50genesPaper_SPM.txt'
top50genesPaper_CNA = '/content/drive/My Drive/Proj Senior/top50genesPaper_CNA.txt'

# Dataset
spm_path = "/content/drive/My Drive/Proj Senior/SomaticPointMutationMatrix.csv"
cna_path = "/content/drive/My Drive/Proj Senior/CopyNumberAlterationMatrix.csv"

In [0]:
# Load Drive
from google.colab import drive
drive.mount('/content/drive')

# Load libraries
%reload_ext autoreload
%autoreload 2
%matplotlib inline

!pip install catboost
import pandas as pd
import time
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
import lightgbm as lgbm
from catboost import CatBoostClassifier
from catboost import core

In [0]:
# Data Preaparation
topPaperSPM = []
topPaperCNA = []
for line in open(top50genesPaper_SPM):
  topPaperSPM.append(line.strip())
for line in open(top50genesPaper_CNA):
  topPaperCNA.append(line.strip())

top50paper = topPaperSPM+topPaperCNA

def count_gene(features):
  CNA = set()
  SPM = set()
  READY = set()
  for gene in features:
    if '.n' in gene:
      CNA.add(gene.replace('.n',''))
      READY.add(gene)
      READY.add(gene.replace('.n','.p'))
    elif '.p' in gene:
      CNA.add(gene.replace('.p',''))
      READY.add(gene)
      READY.add(gene.replace('.p','.n'))
    else:
      READY.add(gene)
      SPM.add(gene)
  print('spm: {}'.format(len(SPM)))
  print('cna: {}'.format(len(CNA)))
  print('gene: {}'.format(len(CNA.union(SPM))))
  return READY

spm = pd.read_csv(spm_path)
cna = pd.read_csv(cna_path)
merged_df = pd.merge(spm,cna,on='Unnamed: 0')
merged_df = merged_df.drop(columns=['classLabels_y'])
merged_df.drop('Unnamed: 0',axis=1, inplace=True)
features_name_all =  list(merged_df.drop(['classLabels_x'], axis=1).columns) # get all features name

selected_paper_df = merged_df[top50paper+['classLabels_x']]

catboostAllRemove = set(top50paper) - set(['GNA11','NPM1', 'RGPD3', 'GNAQ','RBM4'])
catboostAllRemove = merged_df[list(catboostAllRemove)+['classLabels_x']]

In [0]:
kcv1 = StratifiedKFold(n_splits=5, random_state=108, shuffle=True)

# ===============================================================
# SELECT MODEL
# model = LinearSVC()
# model = XGBClassifier(importance_type = 'cover', n_jobs = 4, verbose_eval = 4) # “gain”, “weight”, “cover”, “total_gain” or “total_cover”.
model = RandomForestClassifier()
# model = AdaBoostClassifier(SVC(probability=True, kernel='linear'), n_estimators=50, random_state=0) # tryna add verbose=1
# model = lgbm.LGBMClassifier()
# model = CatBoostClassifier()

# SELECT Dataset
# features = merged_df.copy() # all genes
features = selected_paper_df # 50 genes
# features = catboostAllRemove # Remove 5 genes

classLabels = range(29)[1:]

feature_names = list(features.drop(['classLabels_x'], axis=1).columns)

X = features.drop(['classLabels_x'], axis=1)
y = features['classLabels_x']

In [0]:
# K-FOLD CROSS VALIDATION =======================================
reports = []

scores = []
evas = []
reps = []

imps = []

START_TIME = time.time()


for train_idx, test_idx in kcv1.split(X,y):
    print('Start split')

    coef_ = []
    X_train, X_test, y_train, y_test = X.iloc[train_idx], X.iloc[test_idx], y.iloc[train_idx], y.iloc[test_idx]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # features.append(model.feature_importances_)
    confs = confusion_matrix(y_test, y_pred)
    # coef_ = model.coef_ # for svm only
    reports.append({
      'scores': accuracy_score(y_test, y_pred, normalize=True, sample_weight=None),
      # 'imp': np.copy(model.feature_importances_),
      # 'shap': model.get_feature_importance(data=X_train,type=core.EFstrType.ShapValues),
      'evas': precision_recall_fscore_support(y_test, y_pred, average=None),
      'reps': classification_report(y_test, y_pred),
      'coef_': coef_,
      'confs': confs,
      'precision_micro':  precision_score(y_test, y_pred, average='micro'),
      'recall_micro': recall_score(y_test, y_pred, average='micro'),
      'f1score_micro': f1_score(y_test, y_pred, average='micro'),
    })
    print('checkpoint: {} min'.format((time.time()-START_TIME)/60))

print('take {} min'.format((time.time()-START_TIME)/60))

In [0]:
# Generate report
acc_array = []

precision_array = []
recall_array = []
f1score_array = []
support_array = []

micro_precision_array = []
micro_recall_array = []
micro_f1score_array = []

for rep in reports:
  # print(rep['evas'][0][27])
  precision_array.append(rep['evas'][0])
  recall_array.append(rep['evas'][1])
  f1score_array.append(rep['evas'][2])
  support_array.append(rep['evas'][3])
  micro_precision_array.append(rep['precision_micro'])
  micro_recall_array.append(rep['recall_micro'])
  micro_f1score_array.append(rep['f1score_micro'])
  acc_array.append(rep['scores'])

avg_precision = np.mean(precision_array,axis = 0) 
avg_recall = np.mean(recall_array,axis = 0)
avg_f1score = np.mean(f1score_array, axis = 0)
avg_support = np.mean(support_array, axis=0)
std_precision = np.std(precision_array,axis=0)
std_recall = np.std(recall_array,axis=0)
std_f1score = np.std(f1score_array,axis=0)
std_support = np.std(support_array,axis=0)
item = {
    'label': classLabels,
    'precision': avg_precision,    
    'recall': avg_recall,
    'f1score': avg_f1score,
    'support': avg_support,
    'std_precision': std_precision,
    'std_recall': std_recall,
    'std_f1score': std_f1score,
    'std_support': std_support,    
}

# pd.DataFrame(item).to_excel('/content/drive/My Drive/Proj Senior/Reports/AfterProgress/Rep/CAT50.xlsx')
macro_avg_precision = np.mean(avg_precision)
macro_avg_recall = np.mean(avg_recall)
macro_avg_f1 = 2 * (macro_avg_precision * macro_avg_recall) / (macro_avg_precision + macro_avg_recall)
micro_precision = np.mean(micro_precision_array)
micro_recall = np.mean(micro_recall_array)
micro_f1score = np.mean(micro_f1score_array)
acc = np.mean(acc_array)
print('''macro\tprecision: {}\trecall: {}\tf1score: {}
micro\tprecision: {}\trecall: {}\tf1score: {}
acc: {}'''.format(macro_avg_precision,macro_avg_recall,macro_avg_f1,micro_precision,micro_recall,micro_f1score,acc))
print('std acc: {}'.format(np.std([e['scores'] for e in reports])))
print([e['scores'] for e in reports])
print('====data info====')
count_gene(feature_names)
print('====END====')
print(acc)
print(macro_avg_precision)
print(macro_avg_recall)
print(macro_avg_f1)
print(micro_precision)
print(micro_recall)
print(micro_f1score)